In [1]:
from typing import List
import torch
from torch import _dynamo as torchdynamo

In [2]:
def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print('my_compiler() called with FX graph:')
    gm.graph.print_tabular()
    return gm.forward  # returns a python callable

In [3]:
@torchdynamo.optimize(my_compiler)
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

In [4]:
for _ in range(100):
    toy_example(torch.randn(10), torch.randn(10))

my_compiler() called with FX graph:
opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    l_a_     L_a_                                                    ()                {}
placeholder    l_b_     L_b_                                                    ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7fbcc3d20dc0>  (l_a_,)           {}
call_function  add      <built-in function add>                                 (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                             (l_a_, add)       {}
call_method    sum_1    sum                                                     (l_b_,)           {}
call_function  lt       <built-in function lt>                                  (sum_1, 0)        {}
output         output   output               